# NYC MTA Turnstile Data Cleaning
___

## 1. Data Description & Purpose
---

We will be analysing NYC MTA Turnstile dataset - which tracks passenger entries & exist from subway stations throughtout NYC.  The data is summarized at a 4 hour interval.  

[Source](http://web.mta.info/developers/turnstile.html)

[Field Descriptions](http://web.mta.info/developers/resources/nyct/turnstile/ts_Field_Description.txt)

The purpose of this analysis is to identify the busyness of the NYC subway system during the August to October months.  We will do this by identifying total ridership per station per day over the months of August to October (2016) timeframe.

It would have been ideal to summarize data over the span of 3 years, averaging daily ridership over the 3 years to eliminate any annomalies.  However, initial observations of the data did not allow this for several reasons:
* a) Beginning mid-October 2014, the MTA changed the way they collected turnstile data.  As a result, the datasets differ before & after October 2014 - therefore 2014 was dropped.
* b) Station names appear to different between the 2015 & 2016 datasets (it appears nomenclature was changed at some point - 'St' versus 'Street', 'Av' versus 'Ave' for example).  A mapping of station names could be created, but for the time constraints around this project, we will not undertake this task - therfore 2015 was dropped. 

As a result, we will only use the August to October 2016 dataset. 

## 2. Data Download
___

We can access the data files from the mta website (link provided above).  I've manually keyed the data files we need in the below function - running this function will auto download the files and place them in the ./data directory.

In [37]:
import wget

# Set the url
url_template = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_%s.txt'

# I've listed the set of files that we need to cover the span of Aug 2017 - Oct 2017
for date in ['160806','160813','160820','160827','160903','160910','160917','160924','161001','161008','161015','161022','161029']:  
    
    # build the url
    url = url_template % date
    
    # Uncomment below to begin download
    #wget.download(url, './data/')
    #print(date, 'file downloaded')

## 3. Import Data

With the data downloaded, we can import each data file into a python structure.  We begin organizing this turnstile data so that we can analyze the total number of passengers that pass through a turnstile each day.  

The dictionary data structure will be as follows:

**key:** turnstile ID

**value:** [[timestamp], [timestamp], [timestamp] .... ]]

Let's create a function to read in a MTA data file, and return the above structure

In [1]:
import csv
from collections import defaultdict

def read_csv(file):
    """
        Accepts an MTA turnstile data file, parses each line, and returns an 
        organized dictionary representing a list of timestamps (value) for each
        turnstile id (key)
    """
    
    # create our empty dictionary
    # defaultdict allows us to return an empty list for indicies that do not yet exist
    data = defaultdict(list)
    
    # loop through the input file
    with open(file, 'r') as csv_file:

        reader = csv.reader(csv_file)

        # update user what we are doing
        print('Reading {} ...'.format(file))

        # for each row in the file
        for i, row in enumerate(reader):

            # we know that the first row is headers, so we skip it
            if i != 0: 
                
                # create turnstile id
                ts_id = tuple(row[:6])
                
                # create timestamp data
                ts_timestamp = row[6:]
                
                # add value to dictionary
                data[ts_id].append(ts_timestamp)
    
    return data

In [2]:
# create the list of the files we are going to use
filenames = [
    './data/turnstile_160806.txt',
    './data/turnstile_160813.txt',
    './data/turnstile_160820.txt',
    './data/turnstile_160827.txt',
    './data/turnstile_160903.txt',
    './data/turnstile_160910.txt',
    './data/turnstile_160917.txt',
    './data/turnstile_160924.txt',
    './data/turnstile_161001.txt',
    './data/turnstile_161008.txt',
    './data/turnstile_161015.txt',
    './data/turnstile_161022.txt',    
    './data/turnstile_161029.txt']
 
    
# import the data into raw_data
raw_data = [read_csv(f) for f in filenames]  

Reading ./data/turnstile_160806.txt ...
Reading ./data/turnstile_160813.txt ...
Reading ./data/turnstile_160820.txt ...
Reading ./data/turnstile_160827.txt ...
Reading ./data/turnstile_160903.txt ...
Reading ./data/turnstile_160910.txt ...
Reading ./data/turnstile_160917.txt ...
Reading ./data/turnstile_160924.txt ...
Reading ./data/turnstile_161001.txt ...
Reading ./data/turnstile_161008.txt ...
Reading ./data/turnstile_161015.txt ...
Reading ./data/turnstile_161022.txt ...
Reading ./data/turnstile_161029.txt ...


In [3]:
# Take a sneak peak at our data so far
sample_data = list(raw_data[0].items())[:1]
#print(sample_data)

## 4. Remove Unneccessary Data Fields
___

At the moment, we have our MTA dataset loaded - which includes fields that we do not need for this analysis.  Let's go ahead and cut the additional fields out. 

We can manipulate the current data so that we only have the timestamp and cumulative passenger count. 

The data structure we are trying to achieve is as follows: 

{turnstile ID:

    [ 
    
        [timestamp, passenger counts],
        
        [timestamp, passenger counts],
        
        [timestamp, passenger counts],
      
        ....
         
    ]
    
},
{turnstile ID:

    [
    
        [timestamp, passenger counts],
        
        [timestamp, passenger counts],
        
        [timestamp, passenger counts],
        
         ....
         
    ]
    
),

...


We will create a function to do this data manipulation: mta_create_timestamp

In [4]:
from dateutil.parser import parse
from datetime import datetime
from collections import defaultdict

def mta_create_timestamp(data):
    """
        Accepts a dictionary structure of MTA turnstile data, with the turnstile ID (key) and related 
        time stamped entrires (value).
        
        Manipulates the data to create an actual timestamp for each entry and cumulative passengers. 
    """
    
    # initialize a dictionary to store results
    ts_timestamps = defaultdict(list)
    
    # iterate over the turnstile data
    for i, (turnstile, row_data) in enumerate(data.items()):
        
        # print out our status
        if i % 100 == 0:
            print('Processing turnstile {}...'.format(turnstile))

        # for each time entry, convert to a timestamp and retain cumulative entires
        for datestr, timestr, event, cum_entries, cum_exits in row_data:
            
            # create timestamp
            timestamp = parse(datestr + ' ' + timestr)
            
            # save data back to the dictionary
            ts_timestamps[turnstile].append([timestamp, int(cum_entries)])
    
    return ts_timestamps

In [5]:
# create timestamped data
raw_data_timestamped = [mta_create_timestamp(raw_data[i]) for i in range(len(raw_data))]


Processing turnstile ('A002', 'R051', '02-00-00', '59 ST', 'NQR456', 'BMT')...
Processing turnstile ('A030', 'R083', '01-06-00', '23 ST', 'NR', 'BMT')...
Processing turnstile ('A049', 'R088', '02-01-01', 'CORTLANDT ST', 'R', 'BMT')...
Processing turnstile ('A084', 'R125', '01-00-03', 'BROAD ST', 'JZ', 'BMT')...
Processing turnstile ('B024', 'R211', '00-00-03', 'KINGS HWY', 'BQ', 'BMT')...
Processing turnstile ('C012', 'R258', '01-03-01', '4AV-9 ST', 'DFGMNR', 'BMT')...
Processing turnstile ('E001', 'R368', '00-00-00', '9 AV', 'D', 'BMT')...
Processing turnstile ('H007', 'R248', '00-00-00', '1 AV', 'L', 'BMT')...
Processing turnstile ('H039', 'R375', '00-00-01', 'NEW LOTS', 'L', 'BMT')...
Processing turnstile ('J030', 'R005', '00-00-00', '85 ST-FOREST PK', 'J', 'BMT')...
Processing turnstile ('N012', 'R035', '01-06-03', '168 ST', 'AC1', 'IND')...
Processing turnstile ('N049', 'R084', '01-00-00', '59 ST COLUMBUS', 'ABCD1', 'IND')...
Processing turnstile ('N067', 'R012', '00-03-02', '34 S

In [6]:
# Take a sneak peak at our data so far
sample_data= list(raw_data_timestamped[0].items())[:1]
#print(sample_data)

## 5. Summarize By Turnstile
___

We now have a list of dictionaries - each representing a one week span at MTA.  We want to combine all dictionaries so that we have a complete listing of ridership for each turnstile from August to October.   

We can do this using the combine_weekly_dictionaries function below.

In [7]:
def combine_weekly_dictionaries(data):
    """
        Accepts a list of dictionaries (each representing a two week span in MTA turnstile data), combines
        each dictionary based on turnstile id, and returns the combined dictionary for consumption.
    """

    # declare our variable to hold all data 
    combined = defaultdict(list)
    
    # loop through each dictionary 
    for d in data:
        
        # for each dictionary, loop through k, v pairs (turnstile, timestamps)
        for turnstile, time_series in d.items():
            
            # add the timeseries to the turnstile id
            combined[turnstile] += time_series
            
    return combined

In [8]:
# retrieves the data
timestamped_per_turnstile = combine_weekly_dictionaries(raw_data_timestamped)

In [9]:
# Take a sneak peak at our data so far
list(timestamped_per_turnstile.items())[:1]

[(('A002', 'R051', '02-00-00', '59 ST', 'NQR456', 'BMT'),
  [[datetime.datetime(2016, 7, 30, 0, 0), 5760764],
   [datetime.datetime(2016, 7, 30, 4, 0), 5760814],
   [datetime.datetime(2016, 7, 30, 8, 0), 5760842],
   [datetime.datetime(2016, 7, 30, 12, 0), 5760971],
   [datetime.datetime(2016, 7, 30, 16, 0), 5761196],
   [datetime.datetime(2016, 7, 30, 20, 0), 5761524],
   [datetime.datetime(2016, 7, 31, 0, 0), 5761691],
   [datetime.datetime(2016, 7, 31, 4, 0), 5761733],
   [datetime.datetime(2016, 7, 31, 8, 0), 5761748],
   [datetime.datetime(2016, 7, 31, 12, 0), 5761841],
   [datetime.datetime(2016, 7, 31, 16, 0), 5762029],
   [datetime.datetime(2016, 7, 31, 20, 0), 5762275],
   [datetime.datetime(2016, 8, 1, 0, 0), 5762406],
   [datetime.datetime(2016, 8, 1, 4, 0), 5762419],
   [datetime.datetime(2016, 8, 1, 8, 0), 5762463],
   [datetime.datetime(2016, 8, 1, 12, 0), 5762608],
   [datetime.datetime(2016, 8, 1, 16, 0), 5762961],
   [datetime.datetime(2016, 8, 1, 20, 0), 5763874],
   

## 6. Summarize By Turnstile By Day
___

Great - we now have a single dictionary with a turnstile ID (Key) and the timestamped cumulative passenger entry data (value) per hour.

Now we want to summarize this hourly data into a daily number.  We can do that using our calculate_daily_summary below.

In [10]:
def validate_daily_total(total):
    """
        Takes a total number of passengers that has visited a turnstile over the 
        period of 1 day and validates that its within a predefined limit.  
        For now, the limit is to ensure the total is greater than 0, and less than 10,000 passengers
    """
    if total == None:
        return True
    else:
        return 15000 > total >= 0

In [11]:
from itertools import groupby
from collections import defaultdict

def calculate_daily_summary(data, turnstile_id):
    """
        Takes an array of hourly turnstile entries spread across multiple days and summarizes
        the data at a daily level. 
    """

    # create a varaible to hold our summary per day
    turnstile_daily_summary = []
    error = ''
    
    # create a variable to store the previous day's passenger count
    passenger_count_prevday = None
    
    # group the turnstile entries by date
    for current_date, daily_entries in groupby(data, lambda x: x[0].date()):
        
        # find the maximum for the day
        passenger_count_curday = max([count for time, count in daily_entries])

        # calculate total for the day
        if passenger_count_prevday == None:
            daily_total = None;
        else:
            daily_total = passenger_count_curday - passenger_count_prevday
            
        # check to ensure date count isn't crazy insane
        if validate_daily_total(daily_total):
            turnstile_daily_summary.append([current_date, daily_total])
        else:
            #print('Warning: Date total appears to be incorrect for {} total: {}'.format(current_date, daily_total))
            control, remote, unit, station, line, division = turnstile_id
            error = '{} {}'.format(station, line)
            turnstile_daily_summary.append([current_date, None])
        
        # update the date counter
        passenger_count_prevday = passenger_count_curday
    
    return turnstile_daily_summary, error
                

In [12]:
# create our new summary variable
timestamped_per_turnstile_per_day = defaultdict(list)

# we also want to track any stations that may have an error
errored_entries = set()

# loop through the dictionary and summarize
for i, (turnstile, hourly_entries) in enumerate(timestamped_per_turnstile.items()):
    #print('===   Processing {}   ==='.format(turnstile))
    timestamped_per_turnstile_per_day[turnstile], error = calculate_daily_summary(hourly_entries, turnstile)
    
    # if we find an error - record the station that occurred
    if error:
        errored_entries.add(error)
    

In [13]:
# have a look at the number of stations that potentially have errors
print(len(errored_entries))

129


In [14]:
# Take a sneak peak at our data so far
sample_data = list(timestamped_per_turnstile_per_day.items())[:1]
#print(sample_data)

Based on the above dataset, we can see that a large number of our stations have potential erros in their turnstiles.  Based on the large majority of turnstiles having errors, we will proceed without removing any.

## 7. Summarize Per Station By Day

We now have a daily summary for the number of passengers that visit each turnstile in a station.  A station may have many turnstiles - in order for us to be able to do an analysis, we want to know how many passengers pass through a station.  Let's get the daily summary per station using the group_by_station function below. 

*Note: We are grouping by station & line because it is possible to have the same station name on multiple lines*

In [15]:
def get_station_name(turnstile_data):
    """
        Given turnstile data in the form (turnstile, timeseries), 
        unpack the data and return the identifier defined by station, line
    """
    # unpack the data
    turnstile, timeseries = turnstile_data
    control_area, unit, device_id, station, line, div = turnstile

    return (tuple([station, line, div]))

In [16]:
from collections import Counter
import operator

def group_by_station(data):
    """
        Given data grouped by turnstile, summarize daily passenger counts
        by station.
    """

    mta_data_by_station = {}

    # sort our dictionary first - this ensures we groupby correctly
    data_sorted = sorted(data.items(), key=operator.itemgetter(0))
    
    # loop through data by station
    for station, data in groupby(data_sorted, get_station_name):   
            
        # initalize counter
        daily_counter = Counter()
        
        # loop through each turnstile in the station
        for turnstile, timeseries in data:
                    
            # Unpack the timeseries - begin to summarize by station
            for datestamp, passenger_count in timeseries:           
                
                # ensure we only add integer values
                if passenger_count is not None:
                    daily_counter[datestamp] += passenger_count
                           
        # store the count per station
        mta_data_by_station[station] = sorted(daily_counter.items())
            
    return mta_data_by_station

In [17]:
timestamped_per_station_per_day = group_by_station(timestamped_per_turnstile_per_day)

In [18]:
# Print out all station names
#for st, val in timestamped_per_station_per_day.items():
#    station, line, division = st
#    print('{},{},{}'.format(station, line, division))

## 8. Data Export

We now have cleaned data to a point where we have a timestamp, station name, total ridership.  Let's export this to a csv so that it can be consumed into a dataframe.

In [19]:
import csv

# write to file named nyc_mta_data_cleaned
with open('./data/nyc_mta_data_cleaned.csv', 'w') as f:
    writer = csv.writer(f)
    
    # add header row
    row = []
    row.append('date')
    row.append('station')
    row.append('line')
    row.append('division')
    row.append('daily_ridership')
    writer.writerow(row)

    # loop through each record
    for station, time_entry in timestamped_per_station_per_day.items():
        st, line, div = station

        # add the details of each record to the csv
        for date, ridership in time_entry:
            row = []
            row.append(date)
            row.append(st)
            row.append(line)
            row.append(div)
            row.append(ridership)

            writer.writerow(row)